In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Carregando Pacotes

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
from tensorboard.plugins.hparams import api as hp
from pathlib import Path
import sklearn.metrics
import sklearn
import itertools
import io
import tensorflow_addons as tfa

2022-05-15 17:28:31.144133: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
print(tf.__version__)

2.4.1


# Paramêtros

In [4]:
# Classes
num_classes = 8

# Epochs
n_epochs = 112
n_epochs_ini = 100

# Tamanho imagem
img_rows, img_cols = 224, 224

input_shape = (img_rows, img_cols, 3)

#batch_size = 64

strategy = tf.distribute.MirroredStrategy()

BATCH_SIZE_PER_REPLICA = 16
batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA


# seed
seed = 42

path_dataset = Path('/scratch/dpfurtado/backup_home/corais/5_subimage_224_224/dataset/1_train_50/')
path_train = path_dataset/'train'
path_train_dev = path_dataset/'train-dev'

nome_modelo = 'resnet101_imagenet'


logs_dir = os.path.join('./logs/', nome_modelo)

!rm -r {logs_dir}

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


rm: cannot remove './logs/resnet101_imagenet': No such file or directory


2022-05-15 17:28:32.825642: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-15 17:28:32.827940: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-15 17:28:33.909449: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 312.92GiB/s
2022-05-15 17:28:33.910020: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 312.92GiB/s
2022-05-15 17:28:33.910065: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcu

# Carregando Base de Dados

In [5]:
train_dataset = image_dataset_from_directory(
    path_train,
    image_size=(img_rows, img_cols),
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True)
train_dev_concat_dataset = image_dataset_from_directory(
    path_train_dev,
    image_size=(img_rows, img_cols),
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True)

Found 1394 files belonging to 8 classes.


Found 143 files belonging to 8 classes.


In [6]:
class_names =train_dataset.class_names
for i in range(len(class_names)):
    class_names[i] = class_names[i].replace('_', ' ')

In [7]:
train_dataset = train_dataset.concatenate(train_dev_concat_dataset)

In [8]:
train_dataset = train_dataset.with_options(options)

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(
    buffer_size=AUTOTUNE)  # Otimização

In [10]:
run_dir = logs_dir

with strategy.scope():
    # loss
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # métricas
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')]
    data_augmentation = tf.keras.Sequential(
        [tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical')])
    # Keras sequential model with Hyperparameters passed from the argument
    # Keras sequential model with Hyperparameters passed from the argument
    cnn = ResNet101(weights='imagenet', include_top=False,
                    input_shape=input_shape)
    # Para o primeiro treinamento as camadas convolucionais não serão treinadas
    cnn.trainable = False
    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    # Camadas de Batch Normalization em inference mode
    x = cnn(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)
    predictions = Dense(num_classes)(x)
    model = Model(inputs, predictions)
    lr_denso = 1e-4
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=lr_denso, momentum=0.9), loss=loss, metrics=metrics)


def scheduler_denso(epoch, lr):
    lr_0 = lr_denso
    epoch_ini_lr = 20
    if epoch < epoch_ini_lr:
        learning_rate = lr
        with tf.summary.create_file_writer(run_dir + '/metric').as_default():
            tf.summary.scalar(
                'learning rate', data=learning_rate, step=epoch)
        return learning_rate
    else:
        learning_rate = lr_0 * \
            tf.math.exp(-0.005*(epoch-epoch_ini_lr)).numpy()
        with tf.summary.create_file_writer(run_dir + '/metric').as_default():
            tf.summary.scalar(
                'learning rate', data=learning_rate, step=epoch)
        return learning_rate


callbacks_dense = [tf.keras.callbacks.TensorBoard(run_dir, profile_batch=0, write_graph=False),
                   tf.keras.callbacks.LearningRateScheduler(
    scheduler_denso, verbose=0)]
# Training the network
history_dense = model.fit(train_dataset, epochs=n_epochs_ini, verbose=1, callbacks=callbacks_dense)

treinamento_denso = len(history_dense.history['loss'])
with strategy.scope():
    cnn.trainable = True
    # loss
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # métricas
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')]
    lr_full = 1e-5
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=lr_full, momentum=0.9),
        loss=loss,
        metrics=metrics
    )


def scheduler_cnn(epoch, lr):
    lr_0 = lr_full
    learning_rate = lr_0 * \
        tf.math.exp(-0.02*(epoch-treinamento_denso)).numpy()
    with tf.summary.create_file_writer(run_dir + '/metric').as_default():
        tf.summary.scalar(
            'learning rate', data=learning_rate, step=epoch)
    return learning_rate

nome_save_model = nome_modelo

model_dir = os.path.join('./model/', nome_save_model)

callbacks_cnn = [tf.keras.callbacks.TensorBoard(run_dir, profile_batch=0, write_graph=False),
                 tf.keras.callbacks.LearningRateScheduler(
                 scheduler_cnn, verbose=0)]

history_cnn = model.fit(train_dataset, epochs=n_epochs, initial_epoch=treinamento_denso, verbose=1, callbacks=callbacks_cnn)

treinamento_cnn = len(history_cnn.history['loss'])


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

46/49 [===========================>..] - ETA: 1s - loss: 0.1518 - accuracy: 0.9556

47/49 [===========================>..] - ETA: 1s - loss: 0.1521 - accuracy: 0.9554

48/49 [============================>.] - ETA: 0s - loss: 0.1525 - accuracy: 0.9553

49/49 [==============================] - ETA: 0s - loss: 0.1528 - accuracy: 0.9551

49/49 [==============================] - 29s 582ms/step - loss: 0.1532 - accuracy: 0.9549


In [11]:
print(treinamento_denso)
print(treinamento_cnn)

100
12


In [12]:
model.save(model_dir)

2022-05-15 17:49:30.310075: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./model/resnet101_imagenet/assets
